In [1]:
import pandas as pd
import numpy as np
import pyarrow 
import seaborn as sns



### 1. Extracción de datos
Leer el archivo reservas_hoteles.parquet para obtener la información relevante de hoteles de la competencia.

In [2]:
df_eraw = pd.read_parquet('../data/reservas_hoteles.parquet')
df_eraw

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,cc50c0c1-a589-44b7-8a41-2c731d033f89,Victoriano,Miralles,victoriano.miralles@example.com,False,2025-02-02,2025-03-01,2025-03-02,36,367.65,Hotel Las Estrellas,3.0,Madrid
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2bf0215f-7f7b-4cfb-9a37-d2022c492410,Anacleto,Toledo,anacleto.toledo@example.com,False,2025-02-07,2025-03-01,2025-03-02,38,144.35,Gran Hotel Madrid,4.0,Madrid
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,4ba34638-c43e-455f-886d-b3e3dd223a07,Máxima,Mayoral,máxima.mayoral@example.com,False,2025-02-08,2025-03-01,2025-03-02,29,463.32,Hotel Jardines del Rey,2.0,Madrid
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,a017cb9c-160b-4b90-b4a9-4bdb2dce546a,Rafa,Alcaraz,rafa.alcaraz@example.com,False,2025-02-10,2025-03-01,2025-03-02,15,247.63,Hotel Palacio Imperial,1.0,Madrid


In [3]:
df = df_eraw.copy()

> Utilizamos el atributo df.columns en pandas para conocer el nombre de todas las columnas de una manera rápida

In [4]:
df.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad'],
      dtype='object')

> Obtenemos toda la información del DataFrame para analizarlo

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


Pasamos a formato fecha todas las columnas de fechas que están en otros formatos.

In [6]:
def tran_data(lista_col, dataframe):
    for col in lista_col:
        dataframe[col] = pd.to_datetime(dataframe[col])
    return dataframe

In [7]:
col_fechas = ["fecha_reserva", "inicio_estancia", "final_estancia"]
df = tran_data(col_fechas, df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

Vemos los hoteles únicos en base a si son competencia o no para saber cuántos hoteles tenemos nuestros y cuántos de la competencia (los que sacaremos del web scraping)

In [8]:
df_competencia = df[df["competencia"] == True].drop_duplicates(subset="id_hotel")
df_competencia

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,
7,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,NaT,2025-03-01,2025-03-02,131,NaN,,NaN,
9,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,
11,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,NaT,2025-03-01,2025-03-02,103,NaN,,NaN,
12,8d2d3ea2-14bb-4762-a7a4-a44e75d2d2f4,6c104070-aad0-4372-a6dc-cd38a6f7c3cf,Ricardo,Caballero,ricardo.caballero@example.com,True,NaT,2025-03-01,2025-03-02,181,NaN,,NaN,
16,f4cf7cbf-7a2a-4e44-9b9d-e2c9a444ec78,b1826b3e-e9aa-4a40-b94a-09ad8e7d6507,Lázaro,Pardo,lázaro.pardo@example.com,True,NaT,2025-03-01,2025-03-02,128,NaN,,NaN,
31,4e4f9542-3b51-4da1-b75e-b8c8a2018b6e,85306c06-a7f1-4fdb-807e-66b2178e93c4,Angelina,Priego,angelina.priego@example.com,True,NaT,2025-03-01,2025-03-02,186,NaN,,NaN,
46,7acd5eae-e278-4568-9e35-638532e10b87,43ffdcc5-51b6-4eda-81f5-df2c1ed1adbe,María Belén,Escalona,maría belén.escalona@example.com,True,NaT,2025-03-01,2025-03-02,135,NaN,,NaN,
78,946c7d6c-9523-4056-bc05-4f25b7582202,de5abee1-33ee-421f-9ebd-1baaafafccf9,Rosalva,Cabañas,rosalva.cabañas@example.com,True,NaT,2025-03-01,2025-03-02,117,NaN,,NaN,


In [9]:
df_hoteles_nuestros = df[df["competencia"] == False].drop_duplicates(subset="nombre_hotel")
df_hoteles_nuestros

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid
5,ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7,df42ae3d-0b9b-4c06-b22e-939323833ef8,Joan,Vazquez,joan.vazquez@example.com,False,2025-02-08,2025-03-01,2025-03-02,36,85.45,Hotel Puerta del Cielo,5.0,Madrid
6,99c3dc4f-663c-45f7-849a-ac9313f3746a,a16a9d83-6888-4947-a7ab-ed41203e347c,Chelo,Flor,chelo.flor@example.com,False,2025-02-09,2025-03-01,2025-03-02,43,309.80,Hotel Encanto Real,1.0,Madrid
8,a5da0906-0fe5-4c30-b193-863df67a7b84,4b789497-8da4-4962-b0fa-070556648ed1,Dolores,Lobo,dolores.lobo@example.com,False,2025-02-10,2025-03-01,2025-03-02,6,434.88,Palacio del Sol,4.0,Madrid
10,3eca014e-f122-41d7-9dd3-b984b9153d57,dbaadb0e-a0ff-436e-9b10-c697fb90b720,Noé,Lobo,noé.lobo@example.com,False,2025-02-02,2025-03-01,2025-03-02,3,216.57,Hotel Jardines del Rey,3.0,Madrid
13,8a68fb77-8798-4902-a92d-642ea5e47083,26ca67cd-3762-4f90-9372-c5f3d49df733,Omar,Lamas,omar.lamas@example.com,False,2025-02-10,2025-03-01,2025-03-02,29,378.10,Hotel Las Estrellas,4.0,Madrid
15,5e3de36b-3d2f-4cef-98d3-df826c56a4b1,a263c6e8-96af-4720-96f0-04cfe51116c6,Ágata,Pinedo,ágata.pinedo@example.com,False,2025-02-01,2025-03-01,2025-03-02,47,358.07,Gran Hotel Madrid,2.0,Madrid
17,2a39eb83-b3aa-4df0-93b3-1fdc0b0a9609,b9cea054-2a9a-4772-bc62-feddbd73ea8a,Julie,Baños,julie.baños@example.com,False,2025-02-01,2025-03-01,2025-03-02,14,79.77,Hotel Torre Dorada,3.0,Madrid


In [10]:
df_media_estrellas = round(df_hoteles_nuestros.groupby("nombre_hotel")["estrellas"].mean().reset_index(),1)
df_media_estrellas

,nombre_hotel,estrellas
0,Gran Hotel Madrid,2.0
1,Hotel Brisas del Mar,5.0
2,Hotel Camino del Sol,1.0
3,Hotel Costa Azul,2.0
4,Hotel Encanto Real,1.0
5,Hotel Jardines del Rey,3.0
6,Hotel Las Estrellas,4.0
7,Hotel Los Almendros,5.0
8,Hotel Luz de Madrid,3.0
9,Hotel Maravilla Real,2.0


In [11]:
# generamos un diccionario con los nombres de los hoteles y sus estrellas
nombrehotel=[]
estrellas=[]
for nombre in df_media_estrellas["nombre_hotel"]:
    nombrehotel.append(nombre)
for fila in df_media_estrellas["estrellas"]:
    estrellas.append(fila)
hoteles_estrellas = dict(zip(nombrehotel, estrellas))
hoteles_estrellas

{'Gran Hotel Madrid': 2.0,
 'Hotel Brisas del Mar': 5.0,
 'Hotel Camino del Sol': 1.0,
 'Hotel Costa Azul': 2.0,
 'Hotel Encanto Real': 1.0,
 'Hotel Jardines del Rey': 3.0,
 'Hotel Las Estrellas': 4.0,
 'Hotel Los Almendros': 5.0,
 'Hotel Luz de Madrid': 3.0,
 'Hotel Maravilla Real': 2.0,
 'Hotel Mirador Real': 5.0,
 'Hotel Monte Verde': 1.0,
 'Hotel Palacio Imperial': 2.0,
 'Hotel Puerta del Cielo': 5.0,
 'Hotel Rincón Sereno': 1.0,
 'Hotel Sol y Luna': 1.0,
 'Hotel Torre Dorada': 3.0,
 'Hotel Vista Alegre': 4.0,
 'Palacio del Sol': 4.0}

In [12]:
# Usamos el diccionario para mapear el df de hoteles propios y corregir la columna de estrellas
df_hoteles_nuestros["estrellas"] = df_hoteles_nuestros["nombre_hotel"].map(hoteles_estrellas)
df_hoteles_nuestros

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid
5,ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7,df42ae3d-0b9b-4c06-b22e-939323833ef8,Joan,Vazquez,joan.vazquez@example.com,False,2025-02-08,2025-03-01,2025-03-02,36,85.45,Hotel Puerta del Cielo,5.0,Madrid
6,99c3dc4f-663c-45f7-849a-ac9313f3746a,a16a9d83-6888-4947-a7ab-ed41203e347c,Chelo,Flor,chelo.flor@example.com,False,2025-02-09,2025-03-01,2025-03-02,43,309.80,Hotel Encanto Real,1.0,Madrid
8,a5da0906-0fe5-4c30-b193-863df67a7b84,4b789497-8da4-4962-b0fa-070556648ed1,Dolores,Lobo,dolores.lobo@example.com,False,2025-02-10,2025-03-01,2025-03-02,6,434.88,Palacio del Sol,4.0,Madrid
10,3eca014e-f122-41d7-9dd3-b984b9153d57,dbaadb0e-a0ff-436e-9b10-c697fb90b720,Noé,Lobo,noé.lobo@example.com,False,2025-02-02,2025-03-01,2025-03-02,3,216.57,Hotel Jardines del Rey,3.0,Madrid
13,8a68fb77-8798-4902-a92d-642ea5e47083,26ca67cd-3762-4f90-9372-c5f3d49df733,Omar,Lamas,omar.lamas@example.com,False,2025-02-10,2025-03-01,2025-03-02,29,378.10,Hotel Las Estrellas,4.0,Madrid
15,5e3de36b-3d2f-4cef-98d3-df826c56a4b1,a263c6e8-96af-4720-96f0-04cfe51116c6,Ágata,Pinedo,ágata.pinedo@example.com,False,2025-02-01,2025-03-01,2025-03-02,47,358.07,Gran Hotel Madrid,2.0,Madrid
17,2a39eb83-b3aa-4df0-93b3-1fdc0b0a9609,b9cea054-2a9a-4772-bc62-feddbd73ea8a,Julie,Baños,julie.baños@example.com,False,2025-02-01,2025-03-01,2025-03-02,14,79.77,Hotel Torre Dorada,3.0,Madrid


### Nulos

Vamos a estudiar cómo son los nulos para ver cómo trabajamos con ellos.

In [13]:
df

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,cc50c0c1-a589-44b7-8a41-2c731d033f89,Victoriano,Miralles,victoriano.miralles@example.com,False,2025-02-02,2025-03-01,2025-03-02,36,367.65,Hotel Las Estrellas,3.0,Madrid
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2bf0215f-7f7b-4cfb-9a37-d2022c492410,Anacleto,Toledo,anacleto.toledo@example.com,False,2025-02-07,2025-03-01,2025-03-02,38,144.35,Gran Hotel Madrid,4.0,Madrid
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,4ba34638-c43e-455f-886d-b3e3dd223a07,Máxima,Mayoral,máxima.mayoral@example.com,False,2025-02-08,2025-03-01,2025-03-02,29,463.32,Hotel Jardines del Rey,2.0,Madrid
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,a017cb9c-160b-4b90-b4a9-4bdb2dce546a,Rafa,Alcaraz,rafa.alcaraz@example.com,False,2025-02-10,2025-03-01,2025-03-02,15,247.63,Hotel Palacio Imperial,1.0,Madrid


In [14]:
#Para las fechas de entrada y salida, podemos rellenar toda la columna con la misma fecha ya que son todas iguales.
fecha_reemplazo = "2025-03-01"

df["inicio_estancia"] = df["inicio_estancia"].replace("", None)  # Reemplazar cadenas vacías con None (nulos)
df["inicio_estancia"] = df["inicio_estancia"].fillna(fecha_reemplazo)

# ✅ Convertir la columna a tipo datetime
df["inicio_estancia"] = pd.to_datetime(df["inicio_estancia"])
df["inicio_estancia"].nunique()


1

In [15]:
fecha_reemplazo1 = "2025-03-02"

df["final_estancia"] = df["final_estancia"].replace("", None)  # Reemplazar cadenas vacías con None (nulos)
df["final_estancia"] = df["final_estancia"].fillna(fecha_reemplazo1)

# ✅ Convertir la columna a tipo datetime
df["final_estancia"] = pd.to_datetime(df["final_estancia"])
df["final_estancia"].nunique()

1

In [16]:
df_mail = df["mail"].unique()
df_mail

array(['feliciana.cantón@example.com', 'leonardo.hierro@example.com',
       'maite.calatayud@example.com', ..., 'nadia.verdugo@example.com',
       'custodia.marco@example.com', 'brígida.salom@example.com'],
      shape=(14905,), dtype=object)

In [17]:
# modifica la columna id_cliente generando un id_cliente que comienza en 1 y va asignando cada numero en formato texto a un mail aleatorio
dicc_id_clientes = {}
conteo = 1

for mail in df_mail:
    dicc_id_clientes[mail] = conteo 
    conteo = conteo + 1

df["id"] = df["mail"].map(dicc_id_clientes).astype(str)

print(df["id_cliente"].nunique())
print(df["id_cliente"].dtypes)

df

14847
object


,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,id
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,,1
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,2
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,3
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,4
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,cc50c0c1-a589-44b7-8a41-2c731d033f89,Victoriano,Miralles,victoriano.miralles@example.com,False,2025-02-02,2025-03-01,2025-03-02,36,367.65,Hotel Las Estrellas,3.0,Madrid,3320
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2bf0215f-7f7b-4cfb-9a37-d2022c492410,Anacleto,Toledo,anacleto.toledo@example.com,False,2025-02-07,2025-03-01,2025-03-02,38,144.35,Gran Hotel Madrid,4.0,Madrid,12211
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,4ba34638-c43e-455f-886d-b3e3dd223a07,Máxima,Mayoral,máxima.mayoral@example.com,False,2025-02-08,2025-03-01,2025-03-02,29,463.32,Hotel Jardines del Rey,2.0,Madrid,7155
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,a017cb9c-160b-4b90-b4a9-4bdb2dce546a,Rafa,Alcaraz,rafa.alcaraz@example.com,False,2025-02-10,2025-03-01,2025-03-02,15,247.63,Hotel Palacio Imperial,1.0,Madrid,13302


In [18]:
df

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,id
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,,1
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,2
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,3
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,4
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,cc50c0c1-a589-44b7-8a41-2c731d033f89,Victoriano,Miralles,victoriano.miralles@example.com,False,2025-02-02,2025-03-01,2025-03-02,36,367.65,Hotel Las Estrellas,3.0,Madrid,3320
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2bf0215f-7f7b-4cfb-9a37-d2022c492410,Anacleto,Toledo,anacleto.toledo@example.com,False,2025-02-07,2025-03-01,2025-03-02,38,144.35,Gran Hotel Madrid,4.0,Madrid,12211
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,4ba34638-c43e-455f-886d-b3e3dd223a07,Máxima,Mayoral,máxima.mayoral@example.com,False,2025-02-08,2025-03-01,2025-03-02,29,463.32,Hotel Jardines del Rey,2.0,Madrid,7155
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,a017cb9c-160b-4b90-b4a9-4bdb2dce546a,Rafa,Alcaraz,rafa.alcaraz@example.com,False,2025-02-10,2025-03-01,2025-03-02,15,247.63,Hotel Palacio Imperial,1.0,Madrid,13302


In [19]:
df_mails_unicos = df.drop_duplicates(subset=["mail"])
df_mails_unicos.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,id
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,,1
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,2
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,3
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,4
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,5


In [20]:
df["mail"].nunique()

14905

In [22]:
fecha_reserva = "2025-02-19"

df["fecha_reserva"] = df["fecha_reserva"].replace("", None) #Reemplazar celdas vacías por nulos.
df["fecha_reserva"] = df["fecha_reserva"].fillna(fecha_reserva)
df["fecha_reserva"] = pd.to_datetime(df["fecha_reserva"])

In [24]:
df_hoteles_nuestros = df[df["competencia"] == False].drop_duplicates(subset="id_hotel")
df_hoteles_nuestros

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,id
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,3
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,4
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,5
5,ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7,df42ae3d-0b9b-4c06-b22e-939323833ef8,Joan,Vazquez,joan.vazquez@example.com,False,2025-02-08,2025-03-01,2025-03-02,36,85.45,Hotel Puerta del Cielo,5.0,Madrid,6
6,99c3dc4f-663c-45f7-849a-ac9313f3746a,a16a9d83-6888-4947-a7ab-ed41203e347c,Chelo,Flor,chelo.flor@example.com,False,2025-02-09,2025-03-01,2025-03-02,43,309.80,Hotel Encanto Real,1.0,Madrid,7
8,a5da0906-0fe5-4c30-b193-863df67a7b84,4b789497-8da4-4962-b0fa-070556648ed1,Dolores,Lobo,dolores.lobo@example.com,False,2025-02-10,2025-03-01,2025-03-02,6,434.88,Palacio del Sol,4.0,Madrid,9
10,3eca014e-f122-41d7-9dd3-b984b9153d57,dbaadb0e-a0ff-436e-9b10-c697fb90b720,Noé,Lobo,noé.lobo@example.com,False,2025-02-02,2025-03-01,2025-03-02,3,216.57,Hotel Jardines del Rey,3.0,Madrid,11
13,8a68fb77-8798-4902-a92d-642ea5e47083,26ca67cd-3762-4f90-9372-c5f3d49df733,Omar,Lamas,omar.lamas@example.com,False,2025-02-10,2025-03-01,2025-03-02,29,378.10,Hotel Las Estrellas,4.0,Madrid,14
15,5e3de36b-3d2f-4cef-98d3-df826c56a4b1,a263c6e8-96af-4720-96f0-04cfe51116c6,Ágata,Pinedo,ágata.pinedo@example.com,False,2025-02-01,2025-03-01,2025-03-02,47,358.07,Gran Hotel Madrid,2.0,Madrid,16
18,96a3727c-99fb-4ae0-a6c3-886504499ec2,7164885d-7481-40cc-9aa7-025264d00798,Javier,Mancebo,javier.mancebo@example.com,False,2025-02-12,2025-03-01,2025-03-02,27,228.02,Hotel Palacio Imperial,2.0,Madrid,19


In [23]:
df_competencia = df[df["competencia"] == True].drop_duplicates(subset="id_hotel")
df_competencia

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,id
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-02-19,2025-03-01,2025-03-02,113,NaN,,NaN,,1
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-02-19,2025-03-01,2025-03-02,194,NaN,,NaN,,2
7,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,2025-02-19,2025-03-01,2025-03-02,131,NaN,,NaN,,8
9,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,2025-02-19,2025-03-01,2025-03-02,114,NaN,,NaN,,10
11,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,2025-02-19,2025-03-01,2025-03-02,103,NaN,,NaN,,12
12,8d2d3ea2-14bb-4762-a7a4-a44e75d2d2f4,6c104070-aad0-4372-a6dc-cd38a6f7c3cf,Ricardo,Caballero,ricardo.caballero@example.com,True,2025-02-19,2025-03-01,2025-03-02,181,NaN,,NaN,,13
16,f4cf7cbf-7a2a-4e44-9b9d-e2c9a444ec78,b1826b3e-e9aa-4a40-b94a-09ad8e7d6507,Lázaro,Pardo,lázaro.pardo@example.com,True,2025-02-19,2025-03-01,2025-03-02,128,NaN,,NaN,,17
31,4e4f9542-3b51-4da1-b75e-b8c8a2018b6e,85306c06-a7f1-4fdb-807e-66b2178e93c4,Angelina,Priego,angelina.priego@example.com,True,2025-02-19,2025-03-01,2025-03-02,186,NaN,,NaN,,32
46,7acd5eae-e278-4568-9e35-638532e10b87,43ffdcc5-51b6-4eda-81f5-df2c1ed1adbe,María Belén,Escalona,maría belén.escalona@example.com,True,2025-02-19,2025-03-01,2025-03-02,135,NaN,,NaN,,47
78,946c7d6c-9523-4056-bc05-4f25b7582202,de5abee1-33ee-421f-9ebd-1baaafafccf9,Rosalva,Cabañas,rosalva.cabañas@example.com,True,2025-02-19,2025-03-01,2025-03-02,117,NaN,,NaN,,79


In [50]:
#Eliminamos la columna "id_hotel" del df principal

df = df.drop(columns=["id_hotel"])

In [25]:
df_competencia["id_hotel"]

0     113
1     194
7     131
9     114
11    103
12    181
16    128
31    186
46    135
78    117
Name: id_hotel, dtype: int64

# Gestión DF competencia

In [26]:
#Unimos los datos de la competencia con el df df_competencia y el escrapeo
df_competencia

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,id
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-02-19,2025-03-01,2025-03-02,113,NaN,,NaN,,1
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-02-19,2025-03-01,2025-03-02,194,NaN,,NaN,,2
7,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,2025-02-19,2025-03-01,2025-03-02,131,NaN,,NaN,,8
9,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,2025-02-19,2025-03-01,2025-03-02,114,NaN,,NaN,,10
11,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,2025-02-19,2025-03-01,2025-03-02,103,NaN,,NaN,,12
12,8d2d3ea2-14bb-4762-a7a4-a44e75d2d2f4,6c104070-aad0-4372-a6dc-cd38a6f7c3cf,Ricardo,Caballero,ricardo.caballero@example.com,True,2025-02-19,2025-03-01,2025-03-02,181,NaN,,NaN,,13
16,f4cf7cbf-7a2a-4e44-9b9d-e2c9a444ec78,b1826b3e-e9aa-4a40-b94a-09ad8e7d6507,Lázaro,Pardo,lázaro.pardo@example.com,True,2025-02-19,2025-03-01,2025-03-02,128,NaN,,NaN,,17
31,4e4f9542-3b51-4da1-b75e-b8c8a2018b6e,85306c06-a7f1-4fdb-807e-66b2178e93c4,Angelina,Priego,angelina.priego@example.com,True,2025-02-19,2025-03-01,2025-03-02,186,NaN,,NaN,,32
46,7acd5eae-e278-4568-9e35-638532e10b87,43ffdcc5-51b6-4eda-81f5-df2c1ed1adbe,María Belén,Escalona,maría belén.escalona@example.com,True,2025-02-19,2025-03-01,2025-03-02,135,NaN,,NaN,,47
78,946c7d6c-9523-4056-bc05-4f25b7582202,de5abee1-33ee-421f-9ebd-1baaafafccf9,Rosalva,Cabañas,rosalva.cabañas@example.com,True,2025-02-19,2025-03-01,2025-03-02,117,NaN,,NaN,,79


In [27]:
#Eliminamos las columnas de nombre_hotel, estrellas, ciudad, pais y competencia para evitar duplicados a la hora de hacer el merge.
df_competencia = df_competencia.drop(columns=["nombre_hotel", "estrellas", "precio_noche","fecha_reserva"], axis=1)
df_competencia

,id_reserva,id_cliente,nombre,apellido,mail,competencia,inicio_estancia,final_estancia,id_hotel,ciudad,id
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-03-01,2025-03-02,113,,1
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-03-01,2025-03-02,194,,2
7,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,2025-03-01,2025-03-02,131,,8
9,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,2025-03-01,2025-03-02,114,,10
11,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,2025-03-01,2025-03-02,103,,12
12,8d2d3ea2-14bb-4762-a7a4-a44e75d2d2f4,6c104070-aad0-4372-a6dc-cd38a6f7c3cf,Ricardo,Caballero,ricardo.caballero@example.com,True,2025-03-01,2025-03-02,181,,13
16,f4cf7cbf-7a2a-4e44-9b9d-e2c9a444ec78,b1826b3e-e9aa-4a40-b94a-09ad8e7d6507,Lázaro,Pardo,lázaro.pardo@example.com,True,2025-03-01,2025-03-02,128,,17
31,4e4f9542-3b51-4da1-b75e-b8c8a2018b6e,85306c06-a7f1-4fdb-807e-66b2178e93c4,Angelina,Priego,angelina.priego@example.com,True,2025-03-01,2025-03-02,186,,32
46,7acd5eae-e278-4568-9e35-638532e10b87,43ffdcc5-51b6-4eda-81f5-df2c1ed1adbe,María Belén,Escalona,maría belén.escalona@example.com,True,2025-03-01,2025-03-02,135,,47
78,946c7d6c-9523-4056-bc05-4f25b7582202,de5abee1-33ee-421f-9ebd-1baaafafccf9,Rosalva,Cabañas,rosalva.cabañas@example.com,True,2025-03-01,2025-03-02,117,,79


In [29]:
scrapeo_competencia = pd.read_csv("../data/10_hoteles_competencia.csv")
scrapeo_competencia

,nombre,estrellas,precio_noche,fecha_reserva
0,ibis budget Madrid Centro Lavapies,4.3,106,2025-02-19
1,ibis Styles Madrid Centro Maravillas,4.7,114,2025-02-19
2,ibis budget Madrid Calle 30,4.4,79,2025-02-19
3,ibis Madrid Centro las Ventas,4.5,90,2025-02-19
4,ibis budget Madrid Centro las Ventas,4.3,75,2025-02-19
5,ibis Styles Madrid City Las Ventas,4.6,98,2025-02-19
6,ibis budget Madrid Calle Alcalá,4.3,71,2025-02-19
7,ibis Madrid Calle Alcalá,4.5,78,2025-02-19
8,ibis budget Madrid Vallecas,4.3,70,2025-02-19
9,ibis Madrid Getafe,4.4,64,2025-02-19


In [30]:
#Cambiamos el nombre de la columna "nombre"" por "nombre_hotel" para poder hacer el merge
scrapeo_competencia = scrapeo_competencia.rename(columns={"nombre": "nombre_hotel"})

In [31]:
scrapeo_competencia

,nombre_hotel,estrellas,precio_noche,fecha_reserva
0,ibis budget Madrid Centro Lavapies,4.3,106,2025-02-19
1,ibis Styles Madrid Centro Maravillas,4.7,114,2025-02-19
2,ibis budget Madrid Calle 30,4.4,79,2025-02-19
3,ibis Madrid Centro las Ventas,4.5,90,2025-02-19
4,ibis budget Madrid Centro las Ventas,4.3,75,2025-02-19
5,ibis Styles Madrid City Las Ventas,4.6,98,2025-02-19
6,ibis budget Madrid Calle Alcalá,4.3,71,2025-02-19
7,ibis Madrid Calle Alcalá,4.5,78,2025-02-19
8,ibis budget Madrid Vallecas,4.3,70,2025-02-19
9,ibis Madrid Getafe,4.4,64,2025-02-19


In [32]:
#Añadimos la columna id_hotel a "scrapeo_competencia" para poder hacer el merge
scrapeo_competencia["id_hotel"] = df_competencia["id_hotel"].values
scrapeo_competencia

,nombre_hotel,estrellas,precio_noche,fecha_reserva,id_hotel
0,ibis budget Madrid Centro Lavapies,4.3,106,2025-02-19,113
1,ibis Styles Madrid Centro Maravillas,4.7,114,2025-02-19,194
2,ibis budget Madrid Calle 30,4.4,79,2025-02-19,131
3,ibis Madrid Centro las Ventas,4.5,90,2025-02-19,114
4,ibis budget Madrid Centro las Ventas,4.3,75,2025-02-19,103
5,ibis Styles Madrid City Las Ventas,4.6,98,2025-02-19,181
6,ibis budget Madrid Calle Alcalá,4.3,71,2025-02-19,128
7,ibis Madrid Calle Alcalá,4.5,78,2025-02-19,186
8,ibis budget Madrid Vallecas,4.3,70,2025-02-19,135
9,ibis Madrid Getafe,4.4,64,2025-02-19,117


In [33]:
#Hacemos el merge entre "df_competencia" y "scrapeo_competencia" para completar los datos
df_competencia = df_competencia.merge(scrapeo_competencia, on="id_hotel")
df_competencia

,id_reserva,id_cliente,nombre,apellido,mail,competencia,inicio_estancia,final_estancia,id_hotel,ciudad,id,nombre_hotel,estrellas,precio_noche,fecha_reserva
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-03-01,2025-03-02,113,,1,ibis budget Madrid Centro Lavapies,4.3,106,2025-02-19
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-03-01,2025-03-02,194,,2,ibis Styles Madrid Centro Maravillas,4.7,114,2025-02-19
2,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,2025-03-01,2025-03-02,131,,8,ibis budget Madrid Calle 30,4.4,79,2025-02-19
3,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,2025-03-01,2025-03-02,114,,10,ibis Madrid Centro las Ventas,4.5,90,2025-02-19
4,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,2025-03-01,2025-03-02,103,,12,ibis budget Madrid Centro las Ventas,4.3,75,2025-02-19
5,8d2d3ea2-14bb-4762-a7a4-a44e75d2d2f4,6c104070-aad0-4372-a6dc-cd38a6f7c3cf,Ricardo,Caballero,ricardo.caballero@example.com,True,2025-03-01,2025-03-02,181,,13,ibis Styles Madrid City Las Ventas,4.6,98,2025-02-19
6,f4cf7cbf-7a2a-4e44-9b9d-e2c9a444ec78,b1826b3e-e9aa-4a40-b94a-09ad8e7d6507,Lázaro,Pardo,lázaro.pardo@example.com,True,2025-03-01,2025-03-02,128,,17,ibis budget Madrid Calle Alcalá,4.3,71,2025-02-19
7,4e4f9542-3b51-4da1-b75e-b8c8a2018b6e,85306c06-a7f1-4fdb-807e-66b2178e93c4,Angelina,Priego,angelina.priego@example.com,True,2025-03-01,2025-03-02,186,,32,ibis Madrid Calle Alcalá,4.5,78,2025-02-19
8,7acd5eae-e278-4568-9e35-638532e10b87,43ffdcc5-51b6-4eda-81f5-df2c1ed1adbe,María Belén,Escalona,maría belén.escalona@example.com,True,2025-03-01,2025-03-02,135,,47,ibis budget Madrid Vallecas,4.3,70,2025-02-19
9,946c7d6c-9523-4056-bc05-4f25b7582202,de5abee1-33ee-421f-9ebd-1baaafafccf9,Rosalva,Cabañas,rosalva.cabañas@example.com,True,2025-03-01,2025-03-02,117,,79,ibis Madrid Getafe,4.4,64,2025-02-19


In [34]:
#Por último, rellenamos la columna "ciudad" con el valor "Madrid".capitalizando la primera letra.
df_competencia["ciudad"] = "Madrid"
df_competencia

,id_reserva,id_cliente,nombre,apellido,mail,competencia,inicio_estancia,final_estancia,id_hotel,ciudad,id,nombre_hotel,estrellas,precio_noche,fecha_reserva
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-03-01,2025-03-02,113,Madrid,1,ibis budget Madrid Centro Lavapies,4.3,106,2025-02-19
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-03-01,2025-03-02,194,Madrid,2,ibis Styles Madrid Centro Maravillas,4.7,114,2025-02-19
2,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,2025-03-01,2025-03-02,131,Madrid,8,ibis budget Madrid Calle 30,4.4,79,2025-02-19
3,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,2025-03-01,2025-03-02,114,Madrid,10,ibis Madrid Centro las Ventas,4.5,90,2025-02-19
4,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,2025-03-01,2025-03-02,103,Madrid,12,ibis budget Madrid Centro las Ventas,4.3,75,2025-02-19
5,8d2d3ea2-14bb-4762-a7a4-a44e75d2d2f4,6c104070-aad0-4372-a6dc-cd38a6f7c3cf,Ricardo,Caballero,ricardo.caballero@example.com,True,2025-03-01,2025-03-02,181,Madrid,13,ibis Styles Madrid City Las Ventas,4.6,98,2025-02-19
6,f4cf7cbf-7a2a-4e44-9b9d-e2c9a444ec78,b1826b3e-e9aa-4a40-b94a-09ad8e7d6507,Lázaro,Pardo,lázaro.pardo@example.com,True,2025-03-01,2025-03-02,128,Madrid,17,ibis budget Madrid Calle Alcalá,4.3,71,2025-02-19
7,4e4f9542-3b51-4da1-b75e-b8c8a2018b6e,85306c06-a7f1-4fdb-807e-66b2178e93c4,Angelina,Priego,angelina.priego@example.com,True,2025-03-01,2025-03-02,186,Madrid,32,ibis Madrid Calle Alcalá,4.5,78,2025-02-19
8,7acd5eae-e278-4568-9e35-638532e10b87,43ffdcc5-51b6-4eda-81f5-df2c1ed1adbe,María Belén,Escalona,maría belén.escalona@example.com,True,2025-03-01,2025-03-02,135,Madrid,47,ibis budget Madrid Vallecas,4.3,70,2025-02-19
9,946c7d6c-9523-4056-bc05-4f25b7582202,de5abee1-33ee-421f-9ebd-1baaafafccf9,Rosalva,Cabañas,rosalva.cabañas@example.com,True,2025-03-01,2025-03-02,117,Madrid,79,ibis Madrid Getafe,4.4,64,2025-02-19


In [35]:
df.to_csv("../data/limpieza_reservas_hoteles.csv", index=False)

In [36]:
df_competencia.to_csv("../data/info_competencia.csv", index=False)

In [37]:
df_hoteles_nuestros.to_csv("../data/info_hoteles_nuestros.csv", index=False)